In [21]:
from numpy.linalg import inv, svd
import math

def fit(X, y):
    theta = {}

    U, s, V = svd(X.T)
    D = np.diag(s)
    Z = U.dot(D)
    Sigma_z = Z.dot(Z.T) / len(Z)
    
    for c in np.unique(y):
        prior = np.mean(y == c)
        mu = V.T.dot(np.mean(X[y == c], axis = 0))
        Beta = V.dot(inv(Sigma_z)).dot(mu)
        gamma = mu.T.dot(Beta) + math.log(prior)
        theta[c] = (Beta, gamma)
        
fit(X, y)

ValueError: shapes (210,210) and (2,) not aligned: 210 (dim 1) != 2 (dim 0)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
# from sklearn import qda, lda
import math
from numpy.linalg import inv
import lda
import qda
import lr
import htwt

In [2]:
X, y = htwt.get()

In [3]:
model = lr.fit(X, y)
np.mean(lr.predict(model, X) == y)

0.87142857142857144

In [4]:
model = qda.fit(X, y)
np.mean(qda.predict(model, X) == y)

0.88095238095238093

In [5]:
X = (X - np.mean(X, axis = 0)) / np.std(X, axis = 0)
model= lda.fit(X, y)
np.mean(lda.predict(model, X) == y)

0.87142857142857144